In [1]:
import mmap
import time
from time import strftime, localtime

import import_ipynb
from library import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch_geometric.nn import NNConv, global_mean_pool, GraphUNet, TopKPooling, GCNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.distributions import Normal
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from torch.utils.tensorboard import SummaryWriter
from sys import getsizeof

import glob
import os
import random
import json
import sys
import shutil


torch.set_printoptions(threshold=10_000)
np.set_printoptions(threshold=sys.maxsize)

os.chdir("C:/Users/user/Desktop/suhwan/connection_test/python_agent/stacking/node10/model6")

folderList = glob.glob("history*")

pathName = "history" + str(len(folderList))

print(pathName)


os.mkdir(pathName)



writer = SummaryWriter(pathName)
%matplotlib inline


BUFFER_SIZE = 200000

modelNum = 0
availableJobNum = 0
nodeNum = 0
jobWaitingLength = 0
adjacency = 0

importing Jupyter notebook from library.ipynb
importing Jupyter notebook from job.ipynb
importing Jupyter notebook from jobqueue.ipynb
importing Jupyter notebook from network.ipynb
importing Jupyter notebook from dataplane.ipynb


c:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


history2


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print(torch.cuda.is_available())

print(torch.cuda.get_device_name(device = 0))

device = torch.device('cuda:0')
print(device)

print(type(device))

kkkkk = torch.Tensor(1)

kkkkk = kkkkk.cuda()

print(kkkkk.cuda)
print(type(kkkkk.device))
print(kkkkk.device == device)


True
NVIDIA GeForce RTX 3050
cuda:0
<class 'torch.device'>
<built-in method cuda of Tensor object at 0x000001AE7BC2C1D0>
<class 'torch.device'>
True


In [3]:
learning_rate   = 0.0003 # 0.0003
gamma           = 0.95
entropy_weight  = 0.00001 # 0.001
val_loss_coef = 0.8
'''
entropy_weight : loss함수에 entropy라는 걸 더해주는 테크닉에서의 weight입니다.
이 기법은 A3C 논문에 나와있으며, 쓰는 이유는 exploration을 촉진하기 위해서입니다.
각 액션을 취할 확률을 거의 균등하게끔 업데이트해줌으로써 다양한 액션을 해볼 기회가 많아집니다.
actor-critic에서 이 기법으로 인한 성능 개선이 매우 효과적인 것으로 알고있으므로, 적용해주시면 좋을 듯 합니다.
'''
lmbda         = 0.8
eps_clip      = 0.2
'''
위 두 인자는 PPO에서 씁니다.
'''

batch_size = 128

loss_coef = 0.5

node_feature_num = 100
queue_feature_num = 100
hidden_feature_num = 0

job_generate_rate = 0.003

train_cycle = 300
is_train = True
train_quitient = 0

replay_buffer_size = 10000
history_learning_time = 6

if not is_train:
    train_quitient = train_cycle
    

# 통신 관련 초기화

In [4]:
# import os.path
# import os
# FIFO_FILENAME = './fifo-test'

# if not os.path.exists(FIFO_FILENAME):
#     os.mkfifo(FIFO_FILENAME)
#     if os.path.exists(FIFO_FILENAME):
#         fp_fifo = open(FIFO_FILENAME, "rw")

# for i in range(128):
#     fp_fifo.write("Hello,MakeCode\n")
#     fp_fifo.write("")

import sys 
import win32pipe, win32file, pywintypes

# PIPE_NAME = "\\\\.\\pipe\\omnetPipe2"
PIPE_NAME = "\\\\.\\pipe\\worker3"
BUFFER_SIZE = 200000

try:
    pipe = win32pipe.CreateNamedPipe(
        PIPE_NAME,
        win32pipe.PIPE_ACCESS_DUPLEX,
        win32pipe.PIPE_TYPE_MESSAGE | win32pipe.PIPE_READMODE_MESSAGE | win32pipe.PIPE_WAIT,
        1,
        BUFFER_SIZE,
        BUFFER_SIZE,
        0,
        None
    )    
except:
    pass

win32pipe.ConnectNamedPipe(pipe, None)



0

# 통신 관련 함수

In [5]:
def sendOmnetMessage(msg):
    win32file.WriteFile(pipe, msg.encode('utf-8'))
    
def getOmnetMessage():
    response_byte = win32file.ReadFile(pipe, BUFFER_SIZE)
    response_str = response_byte[1].decode('utf-8')
    return response_str

def closePipe():
    win32file.CloseHandle(pipe)

In [6]:
initial_message = getOmnetMessage()

print(initial_message)

networkInfo = json.loads(initial_message)


modelNum = int(networkInfo['modelNum'])
availableJobNum = int(networkInfo['availableJobNum'])
nodeNum = int(networkInfo['nodeNum'])
jobWaitingLength = int(networkInfo['jobWaitingQueueLength'])
adjacency = eval(networkInfo['adjacencyList'])
episode_length = int(networkInfo['episode_length'])
node_capacity = networkInfo['node_capacity']
job_generate_rate = networkInfo['job_generate_rate']

node_feature_num = 2 * (modelNum * availableJobNum)
queue_feature_num = (nodeNum + modelNum) * jobWaitingLength
hidden_feature_num = 10*(node_feature_num + queue_feature_num)

reward_weight = 1/modelNum

sendOmnetMessage("init") # 입력 끝나면 omnet에 전송

print("네트워크 초기화 완료")

info = f"""
노드 개수 : {nodeNum}
네트워크 최대 job 개수 : {availableJobNum}
job 대기 가능 개수 : {jobWaitingLength}
최대 subtask 개수 : {modelNum}
인접 리스트 : {adjacency}
node_feature_num : {node_feature_num}
queue_feature_num : {queue_feature_num}
episode_length : {episode_length}
node_capacity : {node_capacity}
entropy_weight : {entropy_weight}
reward_weight : {reward_weight}
job_generate_rate : {job_generate_rate}
"""
print(info)

with open(f'{pathName}/info.txt', 'w') as f:
    f.write(f'{info}')

{"modelNum":"6", "availableJobNum":"5", "nodeNum":"10", "jobWaitingQueueLength":"15", "adjacencyList":"[[0,1,0,2,0,3,0,6,0,8,1,3,1,4,1,5,1,7,1,8,3,4,4,5,4,6,4,7,5,9,7,9,],[1,0,2,0,3,0,6,0,8,0,3,1,4,1,5,1,7,1,8,1,4,3,5,4,6,4,7,4,9,5,9,7,]]", "episode_length":"100", "job_generate_rate":"30", "node_capacity":"0.030000, 0.090000"}
네트워크 초기화 완료

노드 개수 : 10
네트워크 최대 job 개수 : 5
job 대기 가능 개수 : 15
최대 subtask 개수 : 6
인접 리스트 : [[0, 1, 0, 2, 0, 3, 0, 6, 0, 8, 1, 3, 1, 4, 1, 5, 1, 7, 1, 8, 3, 4, 4, 5, 4, 6, 4, 7, 5, 9, 7, 9], [1, 0, 2, 0, 3, 0, 6, 0, 8, 0, 3, 1, 4, 1, 5, 1, 7, 1, 8, 1, 4, 3, 5, 4, 6, 4, 7, 4, 9, 5, 9, 7]]
node_feature_num : 60
queue_feature_num : 240
episode_length : 100
node_capacity : 0.030000, 0.090000
entropy_weight : 1e-05
reward_weight : 0.16666666666666666
job_generate_rate : 30



In [7]:
class actor_network(nn.Module):
    def __init__(self):
        super(actor_network, self).__init__()
        self.data = [[] for i in range(modelNum + 1)]
        self.history = [[] for i in range(modelNum + 1)]

        self.x_mean = 0
        self.x2_mean = 0
        self.sd = 0
        self.reward_sample_num = 0

        self.step = 0
        #self.entropy_normalize_weight = 1/torch.log(torch.tensor(nodeNum))
        self.entropy_normalize_weight = 1

        
        # print(f"self.adjacency : ", self.adjacency.shape)

        self.pi_mlp1 = nn.Sequential(nn.Linear(1, node_feature_num * int(node_feature_num//2)), nn.ReLU())
        self.pi_s_ecc1 = NNConv(node_feature_num, int(node_feature_num//2), self.pi_mlp1, aggr='mean')

        self.pi_mlp2 = nn.Sequential(nn.Linear(1, int(node_feature_num//2) * int(node_feature_num//2)), nn.ReLU())
        self.pi_s_ecc2 = NNConv(int(node_feature_num//2), int(node_feature_num//2), self.pi_mlp2, aggr='mean')

        self.pi_graph_u_net1 = GraphUNet(int(node_feature_num//2), 50, int(node_feature_num//2), 3, 0.8)
        self.pi_graph_u_net2 = GraphUNet(int(node_feature_num//2), 50, int(node_feature_num//2), 3, 0.8)

        self.pi_backbone = nn.Sequential(
            nn.Linear((int(node_feature_num//2) + queue_feature_num) * 2, hidden_feature_num),
            nn.ReLU(),
            #nn.Linear(hidden_feature_num, hidden_feature_num), # 38부터 적용되는 코드. 38이전은 두 줄 지우고 실행하면됨.
            #nn.ReLU(),
        )

        """
        self.pi_backbone = nn.Sequential(
            nn.Linear((int(node_feature_num//2) + queue_feature_num) * 2, hidden_feature_num),
            nn.ReLU(),
            nn.Linear(hidden_feature_num, hidden_feature_num), # 38부터 적용되는 코드. 38이전은 두 줄 지우고 실행하면됨.
            nn.ReLU(),
            
        )
        """

        self.heads = [nn.Linear(hidden_feature_num, nodeNum + 1) for i in range(modelNum)]

        # prob_fc : 각 액션에 대한 확률.
        # self.pi_prob_fc = nn.Linear(hidden_feature_num, nodeNum + 1) # nodeNum + voidAction


        # prob_fc : 각 액션에 대한 확률.
        self.v_value_fc = nn.Linear(hidden_feature_num, 1)
        
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
              
        
    # policy DNN
    def pi(self, state, sub_index):
        data, job_waiting_feature = state
        node_feature_1, link_feature_1, adjacency_1 = data[0].x, data[0].edge_attr, data[0].edge_index
        node_feature_2, link_feature_2, adjacency_2 = data[1].x, data[1].edge_attr, data[1].edge_index

        job_waiting_feature_1 = job_waiting_feature[0]
        job_waiting_feature_2 = job_waiting_feature[1]

        if job_waiting_feature_2.device == 'cuda':
            link_feature_1 = link_feature_1.cuda()
            adjacency_1 = adjacency_1.cuda()

            link_feature_2 = link_feature_2.cuda()
            adjacency_2 = adjacency_2.cuda()

        
        # =========================================================================
        node_feature_1 = F.relu(self.pi_s_ecc1(node_feature_1, adjacency_1, link_feature_1))
        node_feature_1 = F.relu(self.pi_graph_u_net1(node_feature_1, adjacency_1))

        node_feature_1 = F.relu(self.pi_s_ecc2(node_feature_1, adjacency_1, link_feature_1))
        node_feature_1 = F.relu(self.pi_graph_u_net2(node_feature_1, adjacency_1))

        data_num_1 = len(node_feature_1) // nodeNum
        
        readout_1 = global_mean_pool(node_feature_1, data[0].batch)

        concat_1 = torch.cat([readout_1, job_waiting_feature_1], dim=1) # 여기에 job waiting 벡터 붙이기.

        # =========================================================================
        node_feature_2 = F.relu(self.pi_s_ecc1(node_feature_2, adjacency_2, link_feature_2))
        node_feature_2 = F.relu(self.pi_graph_u_net1(node_feature_2, adjacency_2))

        node_feature_2 = F.relu(self.pi_s_ecc2(node_feature_2, adjacency_2, link_feature_2))
        node_feature_2 = F.relu(self.pi_graph_u_net2(node_feature_2, adjacency_2))

        data_num_2 = len(node_feature_2) // nodeNum
        
        readout_2 = global_mean_pool(node_feature_2, data[0].batch)

        concat_2 = torch.cat([readout_2, job_waiting_feature_2], dim=1) # 여기에 job waiting 벡터 붙이기.

        # =========================================================================

        concat = torch.cat([concat_1, concat_2], dim=1)

        feature_extract = self.pi_backbone(concat)

        # print(feature_extract)

        output = self.heads[sub_index](feature_extract)
        
        prob = F.softmax(output, dim=1)
        
        # 아래는 엔트로피 구하는 과정
        log_prob = F.log_softmax(output, dim=1)
        entropy = - (log_prob * prob).sum(1, keepdim=True)
        
        return prob, entropy, output

    # advantage network
    def v(self, state, sub_index):
        data, job_waiting_feature = state
        node_feature_1, link_feature_1, adjacency_1 = data[0].x, data[0].edge_attr, data[0].edge_index
        node_feature_2, link_feature_2, adjacency_2 = data[1].x, data[1].edge_attr, data[1].edge_index

        job_waiting_feature_1 = job_waiting_feature[0]
        job_waiting_feature_2 = job_waiting_feature[1]

        if job_waiting_feature_2.device == 'cuda':
            link_feature_1 = link_feature_1.cuda()
            adjacency_1 = adjacency_1.cuda()

            link_feature_2 = link_feature_2.cuda()
            adjacency_2 = adjacency_2.cuda()

        
        # =========================================================================
        node_feature_1 = F.relu(self.pi_s_ecc1(node_feature_1, adjacency_1, link_feature_1))
        node_feature_1 = F.relu(self.pi_graph_u_net1(node_feature_1, adjacency_1))

        node_feature_1 = F.relu(self.pi_s_ecc2(node_feature_1, adjacency_1, link_feature_1))
        node_feature_1 = F.relu(self.pi_graph_u_net2(node_feature_1, adjacency_1))

        data_num_1 = len(node_feature_1) // nodeNum
        
        readout_1 = global_mean_pool(node_feature_1, data[0].batch)

        concat_1 = torch.cat([readout_1, job_waiting_feature_1], dim=1) # 여기에 job waiting 벡터 붙이기.

        # =========================================================================
        node_feature_2 = F.relu(self.pi_s_ecc1(node_feature_2, adjacency_2, link_feature_2))
        node_feature_2 = F.relu(self.pi_graph_u_net1(node_feature_2, adjacency_2))

        node_feature_2 = F.relu(self.pi_s_ecc2(node_feature_2, adjacency_2, link_feature_2))
        node_feature_2 = F.relu(self.pi_graph_u_net2(node_feature_2, adjacency_2))

        data_num_2 = len(node_feature_2) // nodeNum
        
        readout_2 = global_mean_pool(node_feature_2, data[0].batch)

        concat_2 = torch.cat([readout_2, job_waiting_feature_2], dim=1) # 여기에 job waiting 벡터 붙이기.

        # =========================================================================

        concat = torch.cat([concat_1, concat_2], dim=1)

        feature_extract = self.pi_backbone(concat)

        value = self.v_value_fc(feature_extract)

        return value
        
    def put_data(self, transition, index):
        self.data[index].append(transition)

    def set_reward(self, reward):
        self.data[-1][3] = reward

    def set_reward(self, reward):
        self.data[-1][3] = reward
        
    def return_link_dict(sel, ad):
        result = {}
        for i in range(len(ad[0])//2):
            result[f'{ad[0][2*i]}{ad[0][2 *(i)+1]}'] = i
            
        return result

    def return_new_mean(self, mean, num, new_data):
        result = (mean * num + new_data) / (num + 1)
        return result
        
    def return_new_sd(self, square_mean, mean):
        result = (square_mean - mean**2)**0.5
        
        return result
        
    def return_normalize_reward(self, reward):
        self.x_mean = self.return_new_mean(self.x_mean, self.reward_sample_num, reward)
        self.x2_mean = self.return_new_mean(self.x2_mean, self.reward_sample_num, reward**2)
        self.sd = self.return_new_sd(self.x2_mean, self.x_mean)
        
        if self.sd == 0:
                z_normalized = 0
        else:  
            z_normalized = (reward - self.x_mean) / self.sd

        self.reward_sample_num += 1
        
        return z_normalized
        
    
    # make_batch, train_net은 맨 위에 코드 기반 링크와 거의 동일합니다.

    def make_batch_history(self, sub_index):
        sample_index = torch.randperm(len(self.history[sub_index]))[:batch_size]

        if len(sample_index) == 0:
            return -1

        sampled_data = []
        for sample_idx in sample_index:
            sampled_data.append(self.history[sub_index][sample_idx])

        b_size = len(sample_index)

        network_1_lst = [None] * b_size
        network_2_lst = [None] * b_size
        next_network_1_lst = [None] * b_size
        next_network_2_lst = [None] * b_size
        job_waiting_1_lst = [None] * b_size
        job_waiting_2_lst = [None] * b_size
        next_job_waiting_1_lst = [None] * b_size
        next_job_waiting_2_lst = [None] * b_size
        a_lst = [None] * b_size
        r_lst = [None] * b_size
        prob_a_lst = [None] * b_size
        sojourn_time_lst = [None] * b_size
        action_mask_lst = [None] * b_size
        subtask_index_lst = [None] * b_size

        for idx, transition in enumerate(sampled_data):
            #self.history.append(transition)
            network, job_waiting, a, r, nxt_network, nxt_job_waiting, prob_a, sojourn_time, action_mask, subtask_index = transition

            r_lst[idx] = [r*reward_weight]

            network_1_lst[idx] = network[0]
            network_2_lst[idx] = network[1]

            job_waiting_1_lst[idx] = job_waiting[0].tolist()
            job_waiting_2_lst[idx] = job_waiting[1].tolist()

            a_lst[idx] = [a]

            next_network_1_lst[idx] = nxt_network[0]
            next_network_2_lst[idx] = nxt_network[1]
            
            next_job_waiting_1_lst[idx] = nxt_job_waiting[0].tolist()
            next_job_waiting_2_lst[idx] = nxt_job_waiting[1].tolist()

            prob_a_lst[idx] = [prob_a]
            sojourn_time_lst[idx] = [sojourn_time * 5]

            action_mask_lst[idx] = action_mask
            subtask_index_lst[idx] = [subtask_index]

            if idx == batch_size - 1:
                self.data[sub_index] = self.data[sub_index][batch_size:]
                break

        else:
            self.data[sub_index] = []
        

        # gnn sample을 배치단위로 inference하려면 이렇게 묶어줘야 함.
        network_1_loader = DataLoader(network_1_lst, batch_size=len(network_1_lst))
        next_network_1_loader = DataLoader(next_network_1_lst, batch_size=len(network_1_lst))
        network_1_batch = next(iter(network_1_loader))
        next_network_1_batch = next(iter(next_network_1_loader))

        network_2_loader = DataLoader(network_2_lst, batch_size=len(network_2_lst))
        next_network_2_loader = DataLoader(next_network_2_lst, batch_size=len(network_2_lst))
        network_2_batch = next(iter(network_2_loader))
        next_network_2_batch = next(iter(next_network_2_loader))

        # print(job_waiting_lst)
        
        # job_waiting = torch.tensor(np.array(job_waiting_lst), dtype=torch.float)
        job_waiting_1 = torch.squeeze(torch.tensor(np.array(job_waiting_1_lst), dtype=torch.float), dim=1)
        job_waiting_2 = torch.squeeze(torch.tensor(np.array(job_waiting_2_lst), dtype=torch.float), dim=1)

        a = torch.tensor(a_lst)
        r = torch.tensor(r_lst, dtype=torch.float)
        sojourn_time = torch.Tensor(sojourn_time_lst)

        next_job_waiting_1 = torch.squeeze(torch.tensor(np.array(next_job_waiting_1_lst), dtype=torch.float), dim=1)
        next_job_waiting_2 = torch.squeeze(torch.tensor(np.array(next_job_waiting_2_lst), dtype=torch.float), dim=1)
        
        prob_a = torch.tensor(prob_a_lst, dtype=torch.float)
        action_mask_lst = torch.tensor(action_mask_lst)

        subtask_index_lst = torch.tensor(subtask_index_lst, dtype=torch.bool)
        
        # self.data = []
        return [network_1_batch, network_2_batch], [job_waiting_1, job_waiting_2], a, r, [next_network_1_batch, next_network_2_batch], [next_job_waiting_1, next_job_waiting_2], prob_a, sojourn_time, action_mask_lst, subtask_index_lst

        
    def make_batch(self, isFirst, sub_index):

        if(isFirst):
            self.data[sub_index] = self.data[sub_index][::-1]

        b_size = min(len(self.data[sub_index]), batch_size)

        network_1_lst = [None] * b_size
        network_2_lst = [None] * b_size
        next_network_1_lst = [None] * b_size
        next_network_2_lst = [None] * b_size
        job_waiting_1_lst = [None] * b_size
        job_waiting_2_lst = [None] * b_size
        next_job_waiting_1_lst = [None] * b_size
        next_job_waiting_2_lst = [None] * b_size
        a_lst = [None] * b_size
        r_lst = [None] * b_size
        prob_a_lst = [None] * b_size
        sojourn_time_lst = [None] * b_size
        action_mask_lst = [None] * b_size
        subtask_index_lst = [None] * b_size

        for idx, transition in enumerate(self.data[sub_index]):
            #self.history.append(transition)
            network, job_waiting, a, r, nxt_network, nxt_job_waiting, prob_a, sojourn_time, action_mask, subtask_index = transition

            r_lst[idx] = [r*reward_weight]

            network_1_lst[idx] = network[0]
            network_2_lst[idx] = network[1]

            job_waiting_1_lst[idx] = job_waiting[0].tolist()
            job_waiting_2_lst[idx] = job_waiting[1].tolist()

            a_lst[idx] = [a]

            next_network_1_lst[idx] = nxt_network[0]
            next_network_2_lst[idx] = nxt_network[1]
            
            next_job_waiting_1_lst[idx] = nxt_job_waiting[0].tolist()
            next_job_waiting_2_lst[idx] = nxt_job_waiting[1].tolist()

            prob_a_lst[idx] = [prob_a]
            sojourn_time_lst[idx] = [sojourn_time * 5]

            action_mask_lst[idx] = action_mask
            subtask_index_lst[idx] = [subtask_index]


            if idx == batch_size - 1:
                self.data[sub_index] = self.data[sub_index][batch_size:]
                break

        else:
            self.data[sub_index] = []
        

        # gnn sample을 배치단위로 inference하려면 이렇게 묶어줘야 함.
        network_1_loader = DataLoader(network_1_lst, batch_size=len(network_1_lst))
        next_network_1_loader = DataLoader(next_network_1_lst, batch_size=len(network_1_lst))
        network_1_batch = next(iter(network_1_loader))
        next_network_1_batch = next(iter(next_network_1_loader))

        network_2_loader = DataLoader(network_2_lst, batch_size=len(network_2_lst))
        next_network_2_loader = DataLoader(next_network_2_lst, batch_size=len(network_2_lst))
        network_2_batch = next(iter(network_2_loader))
        next_network_2_batch = next(iter(next_network_2_loader))

        # print(job_waiting_lst)
        
        # job_waiting = torch.tensor(np.array(job_waiting_lst), dtype=torch.float)
        job_waiting_1 = torch.squeeze(torch.tensor(np.array(job_waiting_1_lst), dtype=torch.float), dim=1)
        job_waiting_2 = torch.squeeze(torch.tensor(np.array(job_waiting_2_lst), dtype=torch.float), dim=1)

        a = torch.tensor(a_lst)
        r = torch.tensor(r_lst, dtype=torch.float)
        sojourn_time = torch.Tensor(sojourn_time_lst)

        next_job_waiting_1 = torch.squeeze(torch.tensor(np.array(next_job_waiting_1_lst), dtype=torch.float), dim=1)
        next_job_waiting_2 = torch.squeeze(torch.tensor(np.array(next_job_waiting_2_lst), dtype=torch.float), dim=1)
        
        prob_a = torch.tensor(prob_a_lst, dtype=torch.float)
        action_mask_lst = torch.tensor(action_mask_lst)
        
        subtask_index_lst = torch.tensor(subtask_index_lst, dtype=torch.bool)
        
        # self.data = []
        return [network_1_batch, network_2_batch], [job_waiting_1, job_waiting_2], a, r, [next_network_1_batch, next_network_2_batch], [next_job_waiting_1, next_job_waiting_2], prob_a, sojourn_time, action_mask_lst, subtask_index_lst
    
    def train_net(self):
        self = self.cuda()
        for head in self.heads:
            head.cuda()
        
        pre_advantage = [0.0] * (modelNum + 1)
        for index in range(modelNum + 1):
            first = True
            while len(self.data[index]) > 0:
                torch.cuda.empty_cache()

                network_batch, job_waiting, a, r, next_network_batch, next_job_waiting, prob_a, sojourn_time, action_mask, subtask_index = self.make_batch(first, index)

                if first:
                    first = False
                
                network_batch_1 = network_batch[0].clone().cuda()
                network_batch_2 = network_batch[1].clone().cuda()

                network_batch = [network_batch_1, network_batch_2]

                job_waiting_1 = job_waiting[0].clone().cuda()
                job_waiting_2 = job_waiting[1].clone().cuda()

                job_waiting = [job_waiting_1, job_waiting_2]

                a = a.clone().cuda()
                r = r.clone().cuda()

                next_network_batch_1 = next_network_batch[0].clone().cuda()
                next_network_batch_2 = next_network_batch[1].clone().cuda()

                next_network_batch = [next_network_batch_1, next_network_batch_2]

                next_job_waiting_1 = next_job_waiting[0].clone().cuda()
                next_job_waiting_2 = next_job_waiting[1].clone().cuda()

                next_job_waiting = [next_job_waiting_1, next_job_waiting_2]

                prob_a = prob_a.clone().cuda()
                sojourn_time = sojourn_time.clone().cuda()
                gamma_gpu = torch.Tensor([gamma]).clone().cuda()
                action_mask = action_mask.clone().cuda()
                subtask_index = subtask_index.clone().cuda()

                for i in range(2):
                    
                    # void
                    next_v = self.v([next_network_batch, next_job_waiting], index)
                    cur_v = self.v([network_batch, job_waiting], index)

                    td_target = r + gamma_gpu * next_v
                    
                    delta = td_target - cur_v
                    delta = delta.detach().to('cpu').numpy()
                    advantage_lst = []
                    advantage = pre_advantage[index]
                    for i, delta_t in enumerate(delta):
                        advantage = gamma_gpu * lmbda * advantage + delta_t[0]
                        advantage_lst.append([advantage])
                    # advantage_lst.reverse()
                    advantage_lst = torch.tensor(advantage_lst, dtype=torch.float).cuda()

                    pre_advantage[index] = advantage

                    v_loss = val_loss_coef * F.smooth_l1_loss(cur_v , td_target.detach())
                    #print("v_loss", v_loss)
                    
                    self.optimizer.zero_grad()
                    v_loss.mean().backward()
                    self.optimizer.step()


                    pi, current_entropy, outputs = self.pi([network_batch, job_waiting], index)

                    current_entropy = self.entropy_normalize_weight * current_entropy

                    # true가 valid action

                    #outputs = outputs.clone().detach()

                    outputs = outputs * action_mask

                    exp_sum = torch.sum(torch.exp(outputs), dim=1) - torch.sum(action_mask.logical_not(), dim=1) # 0인 애들 빼줌
                    exp_sum = exp_sum.view(-1, 1) # 전치행렬

                    outputs_a = outputs.gather(1,a)
                    pi_a = torch.exp(outputs_a) / exp_sum

                    # pi_a = pi.gather(1,a)
                    ratio = torch.exp(torch.min(torch.tensor(88), torch.log(pi_a + 1e-9) - torch.log(prob_a + 1e-9)))  # a/b == exp(log(a)-log(b))

                    surr1 = ratio * advantage_lst
                    surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage_lst
                    pi_loss = - torch.min(surr1, surr2) - entropy_weight * current_entropy


                    self.optimizer.zero_grad()
                    pi_loss.mean().backward()
                    self.optimizer.step()

    def train_net_history(self):

        self = self.cuda()
        for head in self.heads:
            head.cuda()
        # pre_advantage = 0.0
        if len(self.history) <= 0:
            return


        for index in range(modelNum + 1):
            for i in range(history_learning_time):
                torch.cuda.empty_cache()

                temp_return = self.make_batch_history(index)

                if temp_return == -1:
                    return

                network_batch, job_waiting, a, r, next_network_batch, next_job_waiting, prob_a, sojourn_time, action_mask, subtask_index = temp_return

                if len(network_batch) == 0:
                    return
                
                network_batch_1 = network_batch[0].clone().cuda()
                network_batch_2 = network_batch[1].clone().cuda()

                network_batch = [network_batch_1, network_batch_2]

                job_waiting_1 = job_waiting[0].clone().cuda()
                job_waiting_2 = job_waiting[1].clone().cuda()

                job_waiting = [job_waiting_1, job_waiting_2]

                a = a.clone().cuda()
                r = r.clone().cuda()

                next_network_batch_1 = next_network_batch[0].clone().cuda()
                next_network_batch_2 = next_network_batch[1].clone().cuda()

                next_network_batch = [next_network_batch_1, next_network_batch_2]

                next_job_waiting_1 = next_job_waiting[0].clone().cuda()
                next_job_waiting_2 = next_job_waiting[1].clone().cuda()

                next_job_waiting = [next_job_waiting_1, next_job_waiting_2]

                prob_a = prob_a.clone().cuda()
                sojourn_time = sojourn_time.clone().cuda()
                gamma_gpu = torch.Tensor([gamma]).clone().cuda()
                action_mask = action_mask.clone().cuda()
                subtask_index = subtask_index.clone().cuda()

                next_v = self.v([next_network_batch, next_job_waiting], index)
                cur_v = self.v([network_batch, job_waiting], index)

                td_target = r + gamma_gpu * next_v
                v_loss = val_loss_coef * F.smooth_l1_loss(cur_v , td_target.detach())
                #print("v_loss", v_loss)
                
                self.optimizer.zero_grad()
                v_loss.mean().backward()
                self.optimizer.step()

                td_target = r + gamma_gpu * next_v
                delta = td_target - cur_v

                pi, current_entropy, outputs = self.pi([network_batch, job_waiting], index)

                current_entropy = self.entropy_normalize_weight * current_entropy

                # true가 valid action

                #outputs = outputs.clone().detach()

                outputs = outputs * action_mask


                exp_sum = torch.sum(torch.exp(outputs), dim=1) - torch.sum(action_mask.logical_not(), dim=1) # 0인 애들 빼줌
                exp_sum = exp_sum.view(-1, 1) # 전치행렬

                outputs_a = outputs.gather(1,a)
                pi_a = torch.exp(outputs_a) / exp_sum

                # pi_a = pi.gather(1,a)
                ratio = torch.exp(torch.min(torch.tensor(88), torch.log(pi_a + 1e-9) - torch.log(prob_a + 1e-9)))  # a/b == exp(log(a)-log(b))

                surr1 = ratio * delta.detach()
                surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * delta.detach()
                pi_loss = - torch.min(surr1, surr2) - entropy_weight * current_entropy


                self.optimizer.zero_grad()
                pi_loss.mean().backward()
                self.optimizer.step()

In [8]:
def main():
    global adjacency, entropy_weight, learning_rate
    model = actor_network()
    # model.load_state_dict(torch.load("./history30/model.pth")) # -1 , +1
    # model.load_state_dict(torch.load("C:/Users/user/Desktop/suhwan/connection_test/python_agent/stacking/history18/model.pth")) # original
    reward_history = []
    v_history = []
    
    # network topology의 edges(GNN 예제 링크 참고)

    adjacency = torch.tensor(adjacency, dtype=torch.long)
    

    # node_state, link_state = env.get_state()  # 환경으로부터 실제 state를 관측해 옴(OMNeT++에서 얻어온 statistic로 대체되어야 함).
    # node_state = torch.tensor(node_state, dtype=torch.float)
    # link_state = torch.tensor(link_state, dtype=torch.float)
    # job_waiting_state = env.get_job_waiting_vector()

    # network_state = Data(x=node_state, edge_attr=link_state, edge_index=adjacency)

    # for each time step
    step = 1
    episode = 1

    max_reward = 0

    average_reward = 0
    average_reward_num = 0

    temp_history = [[] for i in range(modelNum + 1)]

    isStop = False
    node_selected_num = [0 for i in range(nodeNum)]
    void_selected_num = 0

    pre_state_1 = {}
    pre_state_2 = {}
    
    while True:
        model = model.to('cpu')
        for head in model.heads:
            head.to('cpu')

        msg = getOmnetMessage()
        
        if msg == "action": # omnet의 메세지, state 받으면 됨
            sendOmnetMessage("ok")
            state_1 = getOmnetMessage()
            state_2 = getOmnetMessage()

            if len(state_1) == 0:
                state_1 = state_2
            

            if len(pre_state_1) == 0: # action 시작
                state_1 = json.loads(state_1) # state 받았으므로 action 하면됨.
                state_2 = json.loads(state_2) # state 받았으므로 action 하면됨.
                
            else:
                state_1 = json.loads(state_1)
                pre_state_1['jobWaiting'] = state_1['jobWaiting']
                pre_state_1['sojournTime'] = state_1['sojournTime']
                state_1 = pre_state_1

                state_2 = json.loads(state_2)
                pre_state_2['jobWaiting'] = state_2['jobWaiting']
                pre_state_2['sojournTime'] = state_2['sojournTime']
                state_2 = pre_state_2
            
            sendOmnetMessage("ok") # 답장

            node_waiting_state_1 = np.array(eval(str(state_1['nodeState'])))
            node_processing_state_1 = np.array(eval(state_1['nodeProcessing']))
            link_state_1 = np.array(eval(state_1['linkWaiting']))
            job_waiting_state_1 = np.array(eval(state_1['jobWaiting']))
            activated_job_list_1 = eval(state_1['activatedJobList'])
            isAction_1 = int(state_1['isAction'])
            reward_1 = float(state_1['reward'])
            averageLatency_1 = float(state_1['averageLatency'])
            completeJobNum_1 = int(state_1['completeJobNum'])
            sojournTime_1 = float(state_1['sojournTime'])

            node_waiting_state_2 = np.array(eval(str(state_2['nodeState'])))
            node_processing_state_2 = np.array(eval(state_2['nodeProcessing']))
            link_state_2 = np.array(eval(state_2['linkWaiting']))
            job_waiting_state_2 = np.array(eval(state_2['jobWaiting']))
            activated_job_list_2 = eval(state_2['activatedJobList'])
            isAction_2 = int(state_2['isAction'])
            reward_2 = float(state_2['reward'])
            averageLatency_2 = float(state_2['averageLatency'])
            completeJobNum_2 = int(state_2['completeJobNum'])
            sojournTime_2 = float(state_2['sojournTime'])

            writer.add_scalar("completeJobNum/train", completeJobNum_2 ,step)
            

            # 이 timestep에서 발생한 모든 샘플에 똑같은 보상 적용.
            if averageLatency_2 == -1:
                reward_2 = 0
            else:
                reward_2 = completeJobNum_2
            
            writer.add_scalar("Reward/train", reward_2, step)
                
            first_sample = True
            

            model.history = model.history[-replay_buffer_size:]

            ################ s' 받아와서 저장 #################
            if is_train and len(pre_state_1) == 0:
                for index, historys in enumerate(temp_history):
                    for history in historys:
                        history[3] = reward_2
                        history[4] = network_state
                        history[5] = job_waiting_state
                        model.history[index].append(history)
                        model.put_data(history, index)
            ###################################################

            temp_history = [[] for i in range(modelNum + 1)]

            job_index = int(state_2['jobIndex'])

            #print('sojourn time :', sojournTime)


            node_state_1 = np.concatenate((node_waiting_state_1,node_processing_state_1) ,axis = 1)
            node_state_1 = torch.tensor(node_state_1, dtype=torch.float)

            node_state_2 = np.concatenate((node_waiting_state_2,node_processing_state_2) ,axis = 1)
            node_state_2 = torch.tensor(node_state_2, dtype=torch.float)

            #print(reward)

            link_state_1 = torch.tensor(link_state_1, dtype=torch.float)
            link_state_2 = torch.tensor(link_state_2, dtype=torch.float)

            job_waiting_num = 0
            job_waiting_queue = collections.deque()
            for job in job_waiting_state_2:
                if any(job): # 하나라도 0이 아닌 것 이 있으면 job이 있는것임.
                    job_waiting_num += 1
                    job_waiting_queue.append(job)
            
            job_waiting_state_1 = torch.tensor(job_waiting_state_1, dtype=torch.float).view(1, -1)
            job_waiting_state_2 = torch.tensor(job_waiting_state_2, dtype=torch.float).view(1, -1)
            # print(job_waiting_state)

            network_state_1 = Data(x=node_state_1, edge_attr=link_state_1, edge_index=adjacency)
            network_state_2 = Data(x=node_state_2, edge_attr=link_state_2, edge_index=adjacency)

            network_state = [network_state_1, network_state_2]
            job_waiting_state = [job_waiting_state_1, job_waiting_state_2]


            

            pre_state_1 = state_1
            pre_state_2 = state_2
            
            if average_reward_num == 0:
                average_reward = reward_2
                average_reward_num = 1
            else:
                average_reward = average_reward + (reward_2 - average_reward)/(average_reward_num + 1)
                average_reward_num += 1
                
            if step > 1:
                for i in range(nodeNum):
                    node_tag = "node/" + str(i) + "/train"
                    writer.add_scalar(node_tag, node_selected_num[i], step)

                writer.add_scalar("node/void/train", void_selected_num, step)
                    
                node_selected_num = [0 for i in range(nodeNum)] # node selected num 초기화
                void_selected_num = 0

                if reward_2 != 0:
                    with torch.no_grad():
                        writer.add_scalar("Value/train", torch.mean(model.v([network_state, job_waiting_state], 0)[0]), step)
                

                writer.flush()

            
            
            # print(job_waiting_queue)
            if job_waiting_num == 0:
                isAction_2 = False
                

            if isAction_2:
                job_idx = job_index
                job = job_waiting_queue.popleft()
                src = -1
                dst = 1
                for i in range(nodeNum):
                    if job[i] == -1:
                        src = i
                    if job[i] == 1:
                        dst = i

                if src == -1:
                    src = dst
                    
                #print(f"src : {src}, dst : {dst}")
                #print(job)
                subtasks = job[nodeNum:]
                offloading_vector = []
                temp_data = []
                scheduling_start = False
                # print(subtasks)
                step += 1

                with torch.no_grad():
                    prob, entropy, output = model.pi([network_state, job_waiting_state], 0)

                writer.add_scalar("Entropy/train", entropy, step)
                #print(f'prob : {prob}')

                # isVoid = F.sigmoid(dists[modelNum].sample())

                m = Categorical(prob) 
                node = m.sample().item()
                #print(f'node : {node}')
                
                # void action 실험용
                # node = nodeNum 
                
                
                # void action 뽑으면 void만 업데이트
                if node == nodeNum and not scheduling_start: 
                    prob[0] = torch.Tensor([0] * nodeNum + [1.0])
                    action_mask = [int(not scheduling_start) if i == node else int(scheduling_start) for i in range(nodeNum + 1)]

                    temp_history[modelNum].append([
                        [network_state[0], network_state[1]], 
                        [job_waiting_state[0], job_waiting_state[1]], 
                        node, 0, 
                        [network_state[0], network_state[1]], 
                        [job_waiting_state[0], job_waiting_state[1]],
                        prob[0][node].item(), 
                        0, action_mask, -1]
                    )

                    sendOmnetMessage("void")

                    #print("action finish.")
                    
                    if getOmnetMessage() == "ok":
                        void_selected_num += 1

                else:
                    scheduling_start = True

                if scheduling_start:

                    for index in range(modelNum):
                        
                        with torch.no_grad():
                            prob, entropy, output = model.pi([network_state, job_waiting_state], index)

                        prob = torch.Tensor([F.softmax(output[0][:nodeNum], dim=0).tolist()])
                        prob = torch.cat([prob[0], torch.tensor([0])]) # void action masking
                        prob = torch.Tensor([prob.tolist()]).cuda()


                        m = Categorical(prob[0])
                        node = m.sample().item()


                        action_mask = [int(not scheduling_start) if i == nodeNum else int(scheduling_start) for i in range(nodeNum + 1)]

                        temp_history[index].append([
                        [network_state[0], network_state[1]], 
                        [job_waiting_state[0], job_waiting_state[1]], 
                        node, -1, 
                        [-1, -1], 
                        [-1, -1],
                        prob[0][node].item(), 
                        0, action_mask, index]
                        )

                        offloading_vector.append(node)
                        node_selected_num[node] += 1
                    
                if len(offloading_vector) != 0: # for문을 다 돌면 -> void action 안뽑으면
                    # print(offloading_vector)
                    msg = str(offloading_vector)
                    sendOmnetMessage(msg)
                    
                    #print("action finish.")
                    if(getOmnetMessage() == "ok"):
                        pass

        elif msg == "stop":
            
            sendOmnetMessage("ok")
            pre_state_1 = {}
            pre_state_2 = {}
            
        elif msg == "episode_finish":
            sendOmnetMessage("ok")

            episodic_reward = getOmnetMessage()
            episodic_reward = json.loads(episodic_reward)
            
            finish_num = float(episodic_reward['reward'])
            complete_num = int(episodic_reward['completNum'])
            average_latency = float(episodic_reward['averageLatency'])

            normalized_finish_num = model.return_normalize_reward(finish_num)
            
            writer.add_scalar("EpisodicReward/train", finish_num, episode)
            writer.add_scalar("NormalizedEpisodicReward/train", normalized_finish_num, episode)
            writer.add_scalar("CompleteNum/train", complete_num, episode)
            writer.add_scalar("averageLatency/train", average_latency ,episode)

            episode += 1
            sendOmnetMessage("ok")

            # learning_rate *= 0.995

            if finish_num > max_reward:
                modelPathName = pathName + "/max_model.pth"
                torch.save(model.state_dict(), modelPathName)
                max_reward = finish_num

            writer.add_scalar("AverageReward/train", average_reward, step)
            average_reward = 0
            average_reward_num = 0

            # entropy_weight *= 0.99


            # if len(model.data) > 0 and step % train_cycle == train_quitient:
            #     tm = localtime(time.time())
            #     time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
            #     print(f"[{time_string}] training....")
            #     model.train_net()
            #     modelPathName = pathName + "/model.pth"
            #     torch.save(model.state_dict(), modelPathName)
            #     writer.add_scalar("AverageReward/train", average_reward, step)
            #     average_reward = 0
            #     average_reward_num = 0
            #     tm = localtime(time.time())
            #     time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
            #     print(f"[{time_string}] training complete")
            if is_train:
                tm = localtime(time.time())
                time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                print(f"[{time_string}] training....")
                model.train_net()
                tm = localtime(time.time())
                time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                print(f"[{time_string}] training complete")

                tm = localtime(time.time())
                time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                print(f"[{time_string}] training replay buffer....")
                model.train_net_history()
                tm = localtime(time.time())
                time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                print(f"[{time_string}] training complete")

                if episode % 100 == 0:
                    modelPathName = pathName + f"/model_{episode}.pth"
                
                modelPathName = pathName + "/model.pth"
                torch.save(model.state_dict(), modelPathName)

        
if __name__ == '__main__':
    main()

[2023-01-10 10:33:18 PM] training....
[2023-01-10 10:33:27 PM] training complete
[2023-01-10 10:33:27 PM] training replay buffer....
[2023-01-10 10:34:20 PM] training complete
[2023-01-10 10:34:23 PM] training....
[2023-01-10 10:34:43 PM] training complete
[2023-01-10 10:34:43 PM] training replay buffer....
[2023-01-10 10:35:39 PM] training complete
[2023-01-10 10:35:44 PM] training....
[2023-01-10 10:36:04 PM] training complete
[2023-01-10 10:36:04 PM] training replay buffer....
[2023-01-10 10:37:03 PM] training complete
[2023-01-10 10:37:08 PM] training....
[2023-01-10 10:37:28 PM] training complete
[2023-01-10 10:37:28 PM] training replay buffer....
[2023-01-10 10:38:28 PM] training complete
[2023-01-10 10:38:32 PM] training....
[2023-01-10 10:38:53 PM] training complete
[2023-01-10 10:38:53 PM] training replay buffer....
[2023-01-10 10:39:54 PM] training complete
[2023-01-10 10:39:58 PM] training....
[2023-01-10 10:40:18 PM] training complete
[2023-01-10 10:40:18 PM] training repla